<a href="https://colab.research.google.com/github/justinballas/WorkAutomation/blob/main/BICFR_Snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is used as a repository of various blocks of code to be used in the Best In Class Fund Review (BICFR) workflow. This notebook is broken down into the following sections:

1. Importing the appropriate files

2. Data Cleaning

3. Carrier Subaccount Analysis and Rankings

4. Best in Class Fund Selection

5. Excel Reporting

In [64]:
!pip install xlsxwriter

In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlsxwriter
from glob import glob
%matplotlib inline

1. Importing the appropriate files

In [150]:
#Initiate a list of carriers. this abbreviation will be used to import each file based on the carrier name and will double as the alias for each carrier for use throughout the Python script. 
carriers_list = ['COIL','LCL', 'NWD', 'PL', 'PRT', 'MM']

#Using Glob to import files based on carrier name
files = [glob('{}*'.format(carrier)) for carrier in carriers_list]
#Glob returns a list, so when using list comprehentions it will return a list of lists, we must convert to a flat list.
files = [item for elem in files for item in elem]
#Reading in the CSVs, skipfooter removes the summary statistics from the Morningstar output
files = [pd.read_csv(file, skipfooter=13, engine='python') for file in files]
#Converts to a dictionary to easily call DataFrames for each carrier. Dictionary structure is used for looping through different amounts carriers to increase reusability of scripts.
carrier_dfs = dict(zip(carriers_list, files))

2. Data Cleaning

In [151]:
#removing "\n" anomaly from wrap text.
for carrier, df in carrier_dfs.items():
  df.rename(columns=lambda x: x.replace('\n',''), inplace=True)

In [152]:
#remove funds that are closed to new investment
for carrier in carriers_list:
  temp_df = carrier_dfs[carrier]
  temp_df.drop(temp_df[temp_df['Closed to New Inv']=='Yes'].index, inplace=True)

In [153]:
#Selecting columns used in BICFR
cols_list = ['Name',
 'Morningstar Category',
 'Advisor',
 'Total Ret 1 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 3 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 5 Yr (Mo-End) Base Currency',
 'Total Ret Annlzd 10 Yr (Mo-End) Base Currency',
 'Morningstar Rating Overall',
 'Morningstar Rating 3 Yr',
 'Morningstar Rating 5 Yr',
 'Morningstar Rating 10 Yr',
 'Total Ret % Rank Cat 1 Yr (Mo-End)',
 'Total Ret % Rank Cat 3 Yr (Mo-End)',
 'Total Ret % Rank Cat 5 Yr (Mo-End)',
 'Total Ret % Rank Cat 10 Yr (Mo-End)',
 'Manager Tenure (Average)',
 'Sharpe Ratio 3 Yr (Mo-End) Risk Currency',
 'Prospectus Net Expense Ratio']

for carrier, df in carrier_dfs.items():
  carrier_dfs[carrier] = df.loc[:,cols_list]

In [154]:
#Remove prefixes in Morningstar Category
replace_list = ['US VL Sub ', 'US Iurance ', 'US Fund']

for carrier in carriers_list:
  temp_df = carrier_dfs[carrier]
  for phrase in replace_list:
    temp_df['Morningstar Category'] = temp_df['Morningstar Category'].str.replace(phrase, "")

In [ ]:
#Remove prefixes in fund names
#create a dict frequency table for phgrases in front of -,ybe ones that show up for often are likely unneeded prefixes.

In [ ]:
#Concat into single dataframe with carrier and fund name as a multi index
[df.insert(loc=0, column='Carrier', value=carrier) for carrier, df in carrier_dfs.items()]
df = pd.concat(carrier_dfs.values())
df.set_index(['Carrier', 'Name'], inplace=True)

3. Carrier Subaccount Rankings and Analysis.

In [163]:
#creating a pivot table to aggregate carrier fund data for rankings.
pivot_table = pd.pivot_table(df, index='Carrier', fill_value=np.nan)

In [180]:
#Seperating into tables for different sheets
man_tenure = pd.merge(pivot_table['Manager Tenure (Average)'], pivot_table['Manager Tenure (Average)'].rank(ascending=False), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

sharpe_ratio = pd.merge(pivot_table['Sharpe Ratio 3 Yr (Mo-End) Risk Currency'], pivot_table['Sharpe Ratio 3 Yr (Mo-End) Risk Currency'].rank(ascending=False), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

exp_ratio = pd.merge(pivot_table['Prospectus Net Expense Ratio'], pivot_table['Prospectus Net Expense Ratio'].rank(), how='left', left_index=True, right_index=True,suffixes=('', ' Rank'))

ms_ratings_cols = ['Morningstar Rating 10 Yr', 'Morningstar Rating 5 Yr', 'Morningstar Rating 3 Yr', 'Morningstar Rating Overall']
ms_ratings =  pivot_table[ms_ratings_cols]
for col in ms_ratings_cols:
  ms_ratings["{} Rank".format(col)] = ms_ratings[col].rank(ascending=False)

rank_cat_cols = ['Total Ret % Rank Cat 1 Yr (Mo-End)', 'Total Ret % Rank Cat 3 Yr (Mo-End)', 'Total Ret % Rank Cat 5 Yr (Mo-End)', 'Total Ret % Rank Cat 10 Yr (Mo-End)']
rank_cat = pivot_table[rank_cat_cols]
for col in rank_cat_cols:
  rank_cat["{} Rank".format(col)] = rank_cat[col].rank()

total_ret_cols = ['Total Ret 1 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 3 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 5 Yr (Mo-End) Base Currency', 'Total Ret Annlzd 10 Yr (Mo-End) Base Currency']
total_ret = pivot_table.loc[:, total_ret_cols]

for col in total_ret_cols:
  total_ret["{} Rank".format(col)] = total_ret[col].rank(ascending=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [181]:
total_ret

,Total Ret 1 Yr (Mo-End) Base Currency,Total Ret Annlzd 3 Yr (Mo-End) Base Currency,Total Ret Annlzd 5 Yr (Mo-End) Base Currency,Total Ret Annlzd 10 Yr (Mo-End) Base Currency,Total Ret 1 Yr (Mo-End) Base Currency Rank,Total Ret Annlzd 3 Yr (Mo-End) Base Currency Rank,Total Ret Annlzd 5 Yr (Mo-End) Base Currency Rank,Total Ret Annlzd 10 Yr (Mo-End) Base Currency Rank
Carrier,,,,,,,,
COIL,38.592250,11.724265,12.238955,9.021091,1.0,3.0,1.0,4.0
LCL,33.319308,11.198095,11.031210,8.399826,6.0,6.0,6.0,6.0
MM,35.515934,11.529121,11.496154,8.756207,4.0,5.0,5.0,5.0
NWD,34.737285,11.662945,11.737122,9.081333,5.0,4.0,4.0,3.0
PL,36.574107,11.991696,12.100642,9.231386,2.0,1.0,2.0,2.0
PRT,35.577426,11.857200,12.014592,9.282747,3.0,2.0,3.0,1.0


In [183]:
pivot_table

,Manager Tenure (Average),Morningstar Rating 10 Yr,Morningstar Rating 3 Yr,Morningstar Rating 5 Yr,Morningstar Rating Overall,Prospectus Net Expense Ratio,Sharpe Ratio 3 Yr (Mo-End) Risk Currency,Total Ret % Rank Cat 1 Yr (Mo-End),Total Ret % Rank Cat 10 Yr (Mo-End),Total Ret % Rank Cat 3 Yr (Mo-End),Total Ret % Rank Cat 5 Yr (Mo-End),Total Ret 1 Yr (Mo-End) Base Currency,Total Ret Annlzd 10 Yr (Mo-End) Base Currency,Total Ret Annlzd 3 Yr (Mo-End) Base Currency,Total Ret Annlzd 5 Yr (Mo-End) Base Currency
Carrier,,,,,,,,,,,,,,,
COIL,7.376500,3.444444,3.388060,3.469697,3.492537,1.007000,0.622353,46.375000,44.000000,45.764706,41.835821,38.592250,9.021091,11.724265,12.238955
LCL,6.745328,3.070175,3.208000,3.178862,3.200000,0.682993,0.704841,46.162791,47.429825,44.192000,45.292683,33.319308,8.399826,11.198095,11.031210
MM,7.003981,3.883721,3.593407,3.626374,3.857143,0.813241,0.710659,39.637363,31.494253,38.593407,37.395604,35.515934,8.756207,11.529121,11.496154
NWD,7.401688,4.061538,3.703448,3.775362,4.020690,0.715649,0.710548,41.060000,28.681481,37.363014,35.381295,34.737285,9.081333,11.662945,11.737122
PL,7.483661,3.442105,3.405405,3.435185,3.504505,0.649911,0.695000,45.750000,70.000000,55.750000,49.000000,36.574107,9.231386,11.991696,12.100642
PRT,7.199604,3.977011,3.326531,3.541667,3.755102,0.816931,0.693700,50.485149,29.288889,46.600000,41.836735,35.577426,9.282747,11.857200,12.014592


In [233]:
def freq_table(data, column):
  table = {}
  for item in data[column].unique():
    freq = len(data[data[column]==item])
    table[item] = freq
  table = pd.DataFrame.from_dict(table, orient='index', columns=['Frequency']).sort_values(by='Frequency', ascending=False)
  return table
  

In [ ]:
freq_table(df, df.index[slice(None), 'Name'])

In [ ]:
df['poop'] = df.reset_index()['Name'].str.split("-", 1).apply(lambda x: x[1])